**교차검증과 그리드 서치**  
테스트 세트를 사용해 성능을 확인하다보면 점점 테스트 세트에 맞추는셈  
테스트 세트로 일반화 성능을 올바르게 에측하려면 가능한 테스트 세트를 사용하지 않아야함.  
모델을 만들고 나서 마지막에  딱 한번만 사용하는 것이 좋음  

**검증세트**  
테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 파악하기 어렵다  
그래서 훈련세트를 한번 더 나누어 검증세트를 두는 것  
|훈련세트|검증세트|테스트세트|  
---
|60%|20%|20%|
---


In [26]:
import pandas as pd
wine =pd.read_csv('https://bit.ly/wine_csv_data')

In [27]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [28]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [29]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)

In [30]:
print(sub_input.shape, sub_target.shape)

(4157, 3) (4157,)


In [31]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


위를 보면 과대 적합이 되어있음을 알 수 있음
#교차검증
교차검증을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 확보할 수 있음  

3폴드 교차  
ex)
세트를 a,b,c 로 나누고
|훈련세트(a)|검증세트(b)|테스트세트(c)|  
|훈련세트(b)|검증세트(a)|테스트세트(c)|   
|훈련세트(c)|검증세트(b)|테스트세트(a)|  
각각의 모델 평가  
이후 검증점수 평균을 냄  
훈련세트를 세부분으로 나눠서 교차 검증을 수행하는 것을 3폴드 교차검증 이라 한다.  
통칭 k-폴드 교차 검증이라고도 부름   
k-fold cross validation.  



In [32]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01168942, 0.00797558, 0.00993276, 0.01007104, 0.00760508]), 'score_time': array([0.00196934, 0.0015409 , 0.00150108, 0.00148225, 0.00135255]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


3폴드를 예를 들었지만  
보통은 5폴드 교차 검증이나 10폴드 교차검증을 많이 사용함  
이렇게 하면 데이터의 80~90%까지 훈련에 사용할 수 있음  
검증 세트가 줄어들지만 각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된 점수로 생각이 가능함  

In [33]:
#이름은 test_score이지만 검증 폴드의 점수임
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


교차 검증을 수행하면 입력 모델에서 얻을 수 있는 최상의 검증 점수를 가늠해 볼 수 있습니다.  

한 가지 주의할 점은 cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않습니다.  
우리는 train_test_split()함수로 전체 데이터를 겄은 후 훈련 세트를 준비했기 때문에 따로 섞을 필요가 없었습니다. 하지만 교차 검증을 할 때 훈련세트를 섞으려면 분할기splitter를 지정해야합니다.  

사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해 줍니다.  
cross_validate() 함수는 기본적으로  
회귀 모델일 경우 KFold 분할기를 사용하고  
분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용합니다. 

In [34]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


만약 훈련세트를 섞은 후 10-폴드 교차 검증을 수행하려면 다음과 같이 작성합니다. 

In [35]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# 하이퍼파라미터 튜닝
모델이 학습하는 파라미터를 모델 파라미터라고 부르고  
모델이 학습할 수 없어 사용자가 지정해야하는 파라미터를 하이퍼 파라미터라고 부름  
사이킷런과 같은 머신러닝 라이브러리를 사용할 때  
이런 하이퍼파라미터는 모두 클래스나 메서드의 매개변수로 표현됨  

하이퍼파라미터를 튜닝하는 작업은 어떻게 진행할까?  
우선 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련함  
검증 세트의 점수나 교차 검증으 ㄹ통해서 매개변수를 조금씩 바꿔 봄  
모델마다 적게는 1-2개, 많게는 5-6개의 매개변수를 제공함  
이 매개변수를 바꿔가면서 모델을 훈련하고 교차 검증을 수행하기  

###사람의 개입 없이 하이퍼파라미터 튜닝을 자동으로 수행하는 기술을 'AutoML'이라고 부름  

두 매개변수를 적절히 바꾸어 최적의 값을 찾는 과정임  
매개변수가 많아지면 문제가 더 복잡해짐 파이썬의 for 반복문을 통해 직접 구현할 수 있기도 하지만  
이미 만들어진 도구를 사용하는 게 편리함  
사이킷런에서 제공하는 그리드 서치Grid Search를 사용하면 됨  

##사이킷런의 GridSearchCV클래스는  
하이퍼파라미터 탐색과 교차 검증을 한 번에 수행함  


In [36]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [37]:
#결정 트리 클래스의 객체를 생성하자마자 바로 전달했음
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [38]:
# 훈련 방법은 동일하게 fit()메서드를 호출하면됨, 이 메서드를 호출하면 그리드 서치 객체는 결정 트리 모델 min_impurity_decrease 값을 바꿔가며 총 5번 실행
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

교차 검증에서 최적의 하이퍼파라미터를 찾으면  
전체 훈련 세트로 모델을 다시 만들어야 한다고 했었음  
사이킷런의 그리드 서치는 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련함  


In [39]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [40]:
# 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장되어 있음
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [41]:
# 각 매개변수에서 수행한 교차 검증은 평균 점수는 cv_results_ 속성의 'mean_test_score'키에 저장되어 있음 
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


첫 번째 값이 가장 큰 걸로 보임  
수동으로 고르는 것보다 넘파이 argmax()함수를 사용하는 방법이 있음  
그 다음 인덱스를 사용해 params 키에 저자ㅏㅇ된 매개변수를 출력할 수 있음

In [42]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


##이 과정의 정리  
1. 먼저 탐색할 매개변수를 지정  
2. 그 다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾기  
3. 그리드 서치는 최ㅣ상의 매개변수에서 전체 훈련 세트를 사용해 최종 모델ㅇ르 훈련함  
  이 모델도 그리드 서치 객체에 저장됨  
##복잡한 매개변수 조합 탐색
min_impurity_decrease는 노드 분할을 위한 불순도 감소 최소량을 지정  
여기에 max_depth로 트리의 깊이를 제한  
min_samples_split으로 노드를 나누기 위한 최소 샘플 수도 골라보기

In [43]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)}

np.arange(0.0001, 0.001, 0.0001)  
arange()함수는  
첫 매개변수 값에서 시작해서  
두번째 매개변수에 도달할 때까지 세번째 매개변수를 계속 더한 배열을 만듦  

파이썬의 range(5, 20, 1) 함수도 비슷함 하지만 이 함수는 정수만 이용이 가능함  
이 경우 max_depth를 5에서 20까지 1씩 증가하면서 15개의 값을 만듦.  

min_samples_split는 2에서 100까지 10씩 증가하면서 10개의 값을 만듦

In [44]:
#이 매개변수로 수행할 교차 검증 횟수는 9*15*10
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [45]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [46]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


아쉬운 부분  
탐색할 매개변수의 간격을 0.0001 혹은 1로 설정했지만 이렇게 간격을 둔 것에 특별한 근거가 부족함  
이보다 더 좁은 간격으로 시도해 볼 수도 있음

##랜덤 서치
매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있음  
또 너무 많은 매개변수 조건이 있어 그리드 서치 수행시간이 오래 걸릴 수 있음  
이럴땐 **랜덤 서치 Random Search**를 사용하면 좋음  

랜덤 서치는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링 할 수 있는 확률 분포 객체를 전달함

In [47]:
from scipy.stats import uniform, randint

In [48]:
rgen = randint(0, 10)
rgen.rvs(10)

array([2, 5, 1, 4, 7, 7, 3, 6, 8, 1])

In [49]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 94,  92, 101, 112,  96, 101, 101, 100, 101, 102]))

In [56]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.62643296, 0.86500326, 0.9658265 , 0.44351905, 0.6853712 ,
       0.99001196, 0.20842338, 0.68619012, 0.57975233, 0.08925873])

In [57]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20, 50),
          'min_samples_leaf': randint(1, 25)}

In [58]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f3b85a035b0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f3b859aa460>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f3b859aa910>},
                   random_state=42)

In [59]:
print(gs.best_params_)

{'max_depth': 36, 'min_impurity_decrease': 0.00041435598107632666, 'min_samples_leaf': 4}


In [60]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8697336566224921


In [61]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.8592307692307692
